In [ ]:
! pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset

# Q3

load a streaming dataset of Lithuanian audio samples from the "facebook/voxpopuli" dataset and answer the questions 3, 4, and 5

What is the gender of the 20th Lithuvanian data point in the VoxPopuli dataset?

In [ ]:
ds_id = "facebook/voxpopuli"
ds=load_dataset(ds_id, "lt", split="train", streaming=True, trust_remote_code=True)


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

In [ ]:
def get_subset(ds, size):
    subset=[]
    for i, example in enumerate(ds):
        if i >= size:
            break
        subset.append(example)
    return subset

In [ ]:
lt_sub=get_subset(ds, 25)

# Q3

load a streaming dataset of Lithuanian audio samples from the "facebook/voxpopuli" dataset and answer the questions 3, 4, and 5

What is the gender of the 20th Lithuvanian data point in the VoxPopuli dataset?

# Q4
What is the speaker-id of the 20th Lithuvanian data point in the VoxPopuli dataset?


In [ ]:
# Q3 and Q4 answer
lt_sub[20]

{'audio_id': '20140113-0900-PLENARY-20-lt_20140113-22:13:25_3',
 'language': 15,
 'audio': {'path': 'train_part_0/20140113-0900-PLENARY-20-lt_20140113-22:13:25_3.wav',
  'array': array([-0.0144043 ,  0.0043335 ,  0.00134277, ..., -0.04492188,
         -0.03408813, -0.03613281]),
  'sampling_rate': 16000},
 'raw_text': 'Deja, skirtingų šalių praktikoje ši įmonių valdymo priemonė vis dar nėra tolygiai išvystyta, todėl Europos Sąjungos lygiu būtina ir prasminga šia tema diskutuoti, ieškoti ir siūlyti būdus, kaip situaciją valstybėse narėse keisti, kokie modeliai būtų patys priimtiniausi.',
 'normalized_text': 'deja skirtingų šalių praktikoje ši įmonių valdymo priemonė vis dar nėra tolygiai išvystyta todėl europos sąjungos lygiu būtina ir prasminga šia tema diskutuoti ieškoti ir siūlyti būdus kaip situaciją valstybėse narėse keisti kokie modeliai būtų patys priimtiniausi.',
 'gender': 'female',
 'speaker_id': '96681',
 'is_gold_transcript': True,
 'accent': 'None'}

# Q5

How many data points are there in the Lithuvanian data in the VoxPopuli dataset?


In [ ]:
count = 0
for i, example in enumerate(ds):
    count += 1
count

456

# Q6 - Q10

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm

In [ ]:
en_ds = load_dataset(ds_id, "en", split="train", streaming=True)
ro_ds = load_dataset(ds_id, "ro", split="train", streaming=True)
sp_ds = load_dataset(ds_id, "es", split="train", streaming=True)

In [ ]:
size=200
en_sub=get_subset(en_ds, size)
ro_sub=get_subset(ro_ds, size)
sp_sub=get_subset(sp_ds, size)

In [ ]:
subset=[]
subset.extend(en_sub)
subset.extend(sp_sub)
subset.extend(ro_sub)

In [ ]:
import random
random.seed(10)
random.shuffle(subset)

In [ ]:
# Laod the Wav2Vec processor and model
model_id="facebook/wav2vec2-base-960h"
processor=Wav2Vec2Processor.from_pretrained(model_id)
model=Wav2Vec2Model.from_pretrained(model_id)
model=model.to("cuda:0")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
call_count = 0
def extract_wav2vec_features(batch):
    global call_count
    call_count += 1
    print(call_count)
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(inputs["input_values"].to("cuda:0"), output_hidden_states=True)
    return {"features":outputs}

In [ ]:
torch.cuda.empty_cache()  # Optionally clear the cache

In [ ]:
subset_with_features = []

for i in range(0, len(subset), 200):
    res = [extract_wav2vec_features(example) for example in subset[i:i+200]]
    subset_with_features.extend(res)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
%%time
import pickle
with open("/content/drive/MyDrive/@Degree/Sep-2024/DLP/audio_data.pickle", "wb") as f:
    pickle.dump(subset_with_features, f)

NameError: name 'subset_with_features' is not defined

<function Tensor.to>

In [ ]:
subset_with_features[0]["features"].hidden_states[0].shape

torch.Size([1, 194, 768])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

https://www.kaggle.com/code/iitm21f1002049/dlp-ga5/edit